In [1]:
from __future__ import print_function
from __future__ import division

import numpy as np
import argparse
import random
import shutil
import time
import warnings


import torch
import torch.nn as nn

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim as optim

import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed

import torchvision
from torchvision import datasets, models, transforms
import time
import os
import sys
import copy

In [2]:
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# import torchvision.models as models

In [3]:
save_path='/scratch/by783/DL_Final_models/'+'try'#+'190424_vgg_ae'#args.save
model_name = 'vgg'#args.model
num_epochs = 2 #args.epochs
feature_extract = True # str2bool(args.pretrained)

###################### fixed_params ###################################

num_classes = 1000
loader_image_path='/scratch/by783/DL_Final/ssl_data_96'
loader_batch_size=128

In [4]:
def image_loader(path, batch_size):
    transform = transforms.Compose(
        [
            #transforms.Resize(input_size),
            #transforms.CenterCrop(input_size),
            # use model fitted with the image size, so no need to resize
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            # https://pytorch.org/docs/stable/torchvision/transforms.html
            # [mean],[std] for different channels
        ]
    )
    sup_train_data = datasets.ImageFolder('{}/{}/train'.format(path, 'supervised'), transform=transform)
    sup_val_data = datasets.ImageFolder('{}/{}/val'.format(path, 'supervised'), transform=transform)
    unsup_data = datasets.ImageFolder('{}/{}/'.format(path, 'unsupervised'), transform=transform)
    # source code: https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
    # Main idea:
    data_loader_sup_train = torch.utils.data.DataLoader(
        sup_train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_sup_val = torch.utils.data.DataLoader(
        sup_val_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_unsup = torch.utils.data.DataLoader(
        unsup_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )

    print('sup_train_data.class_to_idx==sup_val_data.class_to_idx: ',
          sup_train_data.class_to_idx == sup_val_data.class_to_idx)

    return data_loader_sup_train, data_loader_sup_val, data_loader_unsup, sup_train_data.class_to_idx

In [5]:
# https://stackoverflow.com/questions/37837682/python-class-input-argument/37837766
# https://github.com/awentzonline/pytorch-cns/blob/master/examples/vggmse.py

class Model_Based_Autoencoder(torch.nn.Module):
    def __init__(self,model_name, pretrained):
        super(Model_Based_Autoencoder, self).__init__()
        if model_name!='vgg':
            sys.stdout.write('Dear, we only support vgg now...')
        
        self.encoder = models.vgg11_bn(pretrained=pretrained).features
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512,512,kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),#de-conv8
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512,512,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#de-conv7
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512,512,kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),#de-conv6
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512,256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#de-conv5
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256,256,kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),#de-conv4
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256,128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#de-conv3
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128,64,kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),#de-conv2
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64,3,kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),#de-conv1
            nn.BatchNorm2d(3),
            nn.Tanh()
        )
    
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
            
    

In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False




def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0



    if model_name != "vgg":
        sys.stdout.write('We only have vgg now!!!')
    else:
        """ VGG11_bn
        """
        model_ft = Model_Based_Autoencoder('vgg', pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft.encoder, feature_extract)

        input_size = 96

    return model_ft, input_size

In [7]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    #unsupervised learning, we do not need train and vals
    since = time.time()
    loss_history=[]
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        ################# train the model on unsupervised data ############
        model.train()
        
        running_loss = 0.0
        
        for inputs, _ in dataloaders['unlabeled']:
            inputs = inputs.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            
            running_loss += loss.item() * inputs.size(0)
            
            loss.backward()
            optimizer.step()
            
        epoch_loss = running_loss / len(dataloaders['unlabeled'].dataset)
        sys.stdout.write('Training time: {:.0f}s'.format( time.time() - since ))
        sys.stdout.write('Training loss: {:.4f}'.format(epoch_loss))
        ################# evaluate the model performance on labeled data ############
        
        model.eval()
        
        eval_loss=0.0
        for inputs, _ in dataloaders['labeled']:
            inputs = inputs.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            eval_loss += loss.item() * inputs.size(0)
            
        epoch_eval_loss = running_loss / len(dataloaders['labeled'].dataset)
        sys.stdout.write('Evaluation time: {:.0f}s'.format( time.time() - since ))        
        sys.stdout.write(' Eval loss: {:.4f}'.format( epoch_eval_loss))
        
        #################
        
        loss_history.append( ( epoch_loss,epoch_eval_loss ) )
        
        
        if epoch_eval_loss < best_loss:
            best_loss = epoch_eval_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            with open(save_path, 'wb') as f:
                torch.save(model, f)
            
        with open(save_path+'_val_acc', 'w') as f:
            for item in loss_history:
                f.write("unlabeled: %s, labeled: %s \n,  " % (item[0],item[1]) )
    
    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return model, loss_history
        

In [8]:
use_pretrained=True

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

In [9]:
model_ft.encoder[0].weight.requires_grad

False

In [10]:
model_ft.decoder[0].weight.requires_grad

True

In [11]:
criterion = nn.MSELoss()

learning_rate=0.001
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=learning_rate, weight_decay=1e-5)


In [12]:
####### load data, input_size is used ####

sys.stdout.write('Begin to load data...')

dataloaders={}

dataloaders['unlabeled'], dataloaders['labeled'], data_loader_unsup, class_to_idx_dict = image_loader(loader_image_path,loader_batch_size)


Begin to load data...sup_train_data.class_to_idx==sup_val_data.class_to_idx:  True


In [13]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            sys.stdout.write("\t{}".format(name))
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            sys.stdout.write("\t{}".format(name))

Params to learn:
	decoder.0.weight	decoder.0.bias	decoder.1.weight	decoder.1.bias	decoder.3.weight	decoder.3.bias	decoder.4.weight	decoder.4.bias	decoder.6.weight	decoder.6.bias	decoder.7.weight	decoder.7.bias	decoder.9.weight	decoder.9.bias	decoder.10.weight	decoder.10.bias	decoder.12.weight	decoder.12.bias	decoder.13.weight	decoder.13.bias	decoder.15.weight	decoder.15.bias	decoder.16.weight	decoder.16.bias	decoder.18.weight	decoder.18.bias	decoder.19.weight	decoder.19.bias	decoder.21.weight	decoder.21.bias	decoder.22.weight	decoder.22.bias

In [14]:
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/1
----------
Evaluation time: 3685s Eval loss: 1.1384

/home/by783/myenv/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Model_Based_Autoencoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1/1
----------


KeyboardInterrupt: 

这里就用了一个GPU 128 batch size 看起来是最好的

#### 128 batch size:
Epoch 0/1
----------
Evaluation time: 3685s Eval loss: 1.1384

#### 256 batch size:
Epoch 0/0
----------
Training time: 1769sEvaluation time: 3560sTraining loss: 1.2155 Eval loss: 1.2155

Epoch 0/1
----------
Training time: 1563sTraining loss: 1.3153


#### 512 batch size:
Epoch 0/0
Training time: 2219sTraining loss: 1.0315Evaluation time: 4134s Eval loss: 1.0315

In [ ]:
print(adsfa)

In [ ]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder.pth')